In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import scipy
import h5py
import json
import os
from pathlib import Path
from quantify_core.data.handling import set_datadir, get_tuids_containing, extract_parameter_from_snapshot, load_snapshot, load_quantities_of_interest, load_dataset, to_gridded_dataset, load_processed_dataset
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel
from sklearn.decomposition import PCA
from SSRO_functions import *
from matplotlib.legend_handler import HandlerTuple
import pandas as pd
import matplotlib as mpl
%matplotlib inline
from matplotlib.colors import PowerNorm, ListedColormap
import matplotlib.gridspec as gridspec

# Set quantify data directory
set_datadir("O:\FPA_RO_Data")

In [ ]:
# Choose colormap
cmap_blue = plt.cm.Blues
cmap_red = plt.cm.Reds
cmap_green = plt.cm.Greens
cmap_purple = plt.cm.Purples
cmap_orange = plt.cm.Oranges

# Get the colormap colors
cmap_blue = cmap_blue(np.arange(plt.cm.Blues.N))
cmap_red = cmap_red(np.arange(plt.cm.Reds.N))
cmap_green = cmap_green(np.arange(plt.cm.Greens.N))
cmap_purple = cmap_purple(np.arange(plt.cm.Purples.N))
cmap_orange = cmap_orange(np.arange(plt.cm.Oranges.N))

# Set alpha
cmap_blue[:,-1] = np.linspace(0, 1, plt.cm.Blues.N)
cmap_red[:,-1] = np.linspace(0, 1, plt.cm.Reds.N)
cmap_green[:,-1] = np.linspace(0, 1, plt.cm.Greens.N)
cmap_purple[:,-1] = np.linspace(0, 1, plt.cm.Purples.N)
cmap_orange[:,-1] = np.linspace(0, 1, plt.cm.Oranges.N)

# Create new colormap
cmap_blue = ListedColormap(cmap_blue)
cmap_red = ListedColormap(cmap_red)
cmap_green = ListedColormap(cmap_green)
cmap_purple = ListedColormap(cmap_purple)
cmap_orange = ListedColormap(cmap_orange)

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
blues = cm.get_cmap('Blues', 10)
greens = cm.get_cmap('Greens', 10)
oranges = cm.get_cmap('Oranges', 10)
purples = cm.get_cmap('Purples', 10)
reds = cm.get_cmap('Reds', 10)

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)

# Get SSRO vs drive amplitude


In [ ]:
tuid_ssro = get_tuids_containing('SSRO FX8', t_start='20231129-220122', t_stop='20231129-221053')

In [ ]:
epsilon = [] #V
snr = []

for i,tuid in enumerate(tuid_ssro):
    snapshot = load_snapshot(tuid)
    pulse_amp = extract_parameter_from_snapshot(snapshot, "FX8.measure_conventional.pulse_amp")['value']
    epsilon.append(pulse_amp)
    _snr, _error = get_paperdef_snr_error(tuid)
    snr.append(_snr)

In [ ]:
#fit with linear curve
p0 = [0.5/0.015] #initial guess
popt,_ = curve_fit(lin_fit, epsilon, snr, p0=p0)

# Get coherence vs drive amplitude

In [ ]:
tuid_coherence = load_processed_dataset(tuid='20231129-203933', analysis_name='RamseyVsPulseAmplitudeAnalysis')
coherence = np.array(tuid_coherence.coherence)
drive_amps = np.array(tuid_coherence.unique_amps)

In [ ]:
#fit with gaussian
p0 = [0.175, 0.05] #initial guess
popt2,_ = scipy.optimize.curve_fit(coherence_decay, drive_amps, coherence, p0=p0)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.19, right=0.9, wspace=0.4, hspace=0.3)

fig.set_size_inches(7/2, 2)
ax1 = ax
# ax2 = ax[1]

tick_fs = 9
label_fs = 10
lw = 1.5

coherence_data=cmap_red(0.9)
coherence_fit=cmap_red(0.6)
snr_data=cmap_blue(0.9)
snr_fit=cmap_blue(0.6)

ax1.set_xlim(epsilon[0]*10**3, epsilon[-1]*10**3)
ax1.set_ylim(0, 0.55)
ax2 = ax1.twinx()
ax1.set_ylabel(r"SNR", color=cmap_blue(0.9), fontsize=label_fs)
ax2.set_ylabel(r"Coherence $|\rho_{10}|$", color=cmap_red(0.9), fontsize=label_fs)
ax1.set_xlabel(r"Drive Amplitude, $\epsilon$ (mV)", fontsize=label_fs)
ax1.set_yticks([0.2, 0.4], labels=['0.2', '0.4'], fontsize=tick_fs)
ax1.set_xticks([0, 5, 10], labels=['0', '5', '10'], fontsize=tick_fs)
ax2.set_yticks([0.05, 0.10, 0.15], labels=['0.05', '0.10', '0.15'], fontsize=tick_fs)
ax1.arrow(x=4, y=0.2, dx=-3, dy=0, width=0.003, head_length=0.2, color=snr_data)
ax1.arrow(x=10.75, y=0.2, dx=3, dy=0, width=0.003, head_length=0.2, color=coherence_data)
a, = ax2.plot(np.array(drive_amps)*10**(3), coherence, linestyle='', marker='.', markersize=6, color=coherence_data)
b, = ax2.plot(np.array(drive_amps)*10**(3), coherence_decay(drive_amps, *popt2), linestyle="-", linewidth=1.5, color=coherence_fit)
c, = ax.plot(np.array(epsilon)*10**(3), lin_fit(np.array(epsilon), *popt), linestyle='-', linewidth=1.5, color=snr_fit)
d, = ax.plot(np.array(epsilon)*10**(3), snr, marker='.', linestyle=' ', markersize=6, color=snr_data)
l = ax2.legend([(a, d), (b, c)], [r"Data", r"Fit"],
               handler_map={tuple: HandlerTuple(ndivide=None)}, handletextpad=0.6, frameon=False, handlelength = 1.4, fontsize = tick_fs, labelspacing=0.3, 
               loc='lower left', bbox_to_anchor=(0.37, -0.05))

# Measurement efficiency and photon number

## Extract $\eta$

In [ ]:
a = popt[0]
sigma = popt2[1]
meas_efficiency = (a**2 * sigma**2)
print(meas_efficiency)

## Extract $\bar{n}$

In [ ]:
df1 = pd.read_csv(r'res_fit_results.csv')

#phi_exts
phi_ext_disshift = df1['phi_ext_disshift'].to_numpy()
phi_ext_disshift = phi_ext_disshift[~np.isnan(phi_ext_disshift)]

#resonator frequencies (GHz)
frs_0_all = df1['fr_q0'].to_numpy()
frs_0_all = frs_0_all[~np.isnan(frs_0_all)]
frs_1_all = df1['fr_q1'].to_numpy()
frs_1_all = frs_1_all[~np.isnan(frs_1_all)]
kappa = df1['kappa_avg'].to_numpy()
kappa = kappa[~np.isnan(kappa)]

#chi (MHz)
chi_exp = df1['chi'].to_numpy()
chi_exp = chi_exp[~np.isnan(chi_exp)]

In [ ]:
total_time = (34/15)+1.52
on_time = (34/15)
off_time = 1.52

meas_strength = 0.4

In [ ]:
meas_strength = [0.1, 0.15, 0.2, 0.3, 0.4]
nbar = []
nmax = []
for i in meas_strength:
    avg_n = (2*np.pi*kappa[0]*i**2)/(32*(sigma**2)*((2*np.pi*chi_exp[0])**2)*total_time)
    max_n = avg_n*(total_time/on_time)
    nbar.append(avg_n)
    nmax.append(max_n)

In [ ]:
meas_strength = 0.4
avg_photons = ((2*np.pi*kappa[0])*meas_strength**2)/(32*(sigma**2)*((2*np.pi*chi_exp[0])**2)*total_time)
max_photons = avg_photons*(total_time/on_time)

In [ ]:
print(nbar)
print(nmax)

# Import and plot flux pulse amp calibration

In [ ]:
df1 = pd.read_csv(r'res_fit_results.csv')

#phi_exts
phi_ext_disshift = df1['phi_ext_disshift'].to_numpy()
phi_ext_disshift = phi_ext_disshift[~np.isnan(phi_ext_disshift)]

#resonator frequencies (GHz)
frs_0_all = df1['fr_q0'].to_numpy()
frs_0_all = frs_0_all[~np.isnan(frs_0_all)]
frs_1_all = df1['fr_q1'].to_numpy()
frs_1_all = frs_1_all[~np.isnan(frs_1_all)]
kappa = df1['kappa_avg'].to_numpy()
kappa = kappa[~np.isnan(kappa)]

#chi (MHz)
chi_exp = df1['chi'].to_numpy()
chi_exp = chi_exp[~np.isnan(chi_exp)]

#resonator frequency vs AC flux pulse amplitude
fp_amp = df1['flux_pulse_amp'].to_numpy()
fp_amp = fp_amp[~np.isnan(fp_amp)]

fr_q0_fp = df1['fr_q0_flux_pulse'].to_numpy()
fr_q0_fp = fr_q0_fp[~np.isnan(fr_q0_fp)]

In [ ]:
fig = plt.figure(figsize=(3.40457, 1.75))

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1.5], wspace=0.35, hspace=0.38, bottom = 0.18, top=0.81, left=0.14, right=0.86,)

label_fs = 7
tick_fs = 7
legend_fs = 6

ax = plt.subplot(gs[0, 0])

ax.set_xlim(0.5, 0.7)
ax2 = ax.twiny()
ax2.set_xlim(0, 0.1155)
ax.set_ylim(5.170, 5.185)

ax.set_yticks([5.17, 5.18], labels=['5.17', '5.18'], fontsize=tick_fs)
ax2.set_xticks([0.0, 0.05, 0.10], labels=['0', '0.05', '0.10'], fontsize=tick_fs)
ax.set_xticks([0.5, 0.6, 0.7], labels=['0.5', '0.6', '0.7'], fontsize=tick_fs)
ax.tick_params(axis='x', pad=1, labelsize=tick_fs)
ax.tick_params(axis='y', pad=1, labelsize=tick_fs)
ax2.tick_params(axis='x', pad=1, labelsize=tick_fs)


ax.set_ylabel(r'$\omega_{r}/2\pi$ (GHz)', color='black', fontsize=label_fs, labelpad=1)
ax.set_xlabel(r"$\Phi_{ext}/\Phi_{0}$", color='black', fontsize=label_fs, labelpad=1)
ax2.set_xlabel('Pulse Amp. (au)', fontsize=label_fs)

ax2.plot(fp_amp, fr_q0_fp*10**(-9), ls='', marker='x', markersize=5, color=cmap_orange(0.6), label='Flux pulse')
ax.plot(phi_ext_disshift, frs_0_all, linestyle='', marker='.', markersize=4, color=cmap_red(0.8), label='DC bias', zorder=2)

ax2.legend(handletextpad=0.3, frameon=False, handlelength = 1.3, fontsize = legend_fs, labelspacing=0.5, 
               loc='lower left', bbox_to_anchor=(-0.02, 0.74))
ax.legend(handletextpad=0.3, frameon=False, handlelength = 1.3, fontsize = legend_fs, labelspacing=0.5, 
               loc='lower left', bbox_to_anchor=(-0.02, 0.83), markerscale=1.5)
ax.annotate("(a)", xy=(-0.24, 0.94), xycoords="axes fraction", fontsize=label_fs)


coherence_data=cmap_red(0.9)
coherence_fit=cmap_red(0.6)
snr_data=cmap_blue(0.9)
snr_fit=cmap_blue(0.6)

ax3 = plt.subplot(gs[0, 1])
ax3.set_xlim(epsilon[0]*10**3, epsilon[-1]*10**3)
ax3.set_ylim(0, 0.55)
ax4 = ax3.twinx()
ax3.set_ylabel(r"SNR", color=cmap_blue(0.9), fontsize=label_fs, labelpad=1)
ax4.set_ylabel(r"Coherence $|\rho_{10}|$", color=cmap_red(0.9), fontsize=label_fs, labelpad=1)
ax3.set_xlabel(r"Drive Amp., $\epsilon$ (mV)", fontsize=label_fs, labelpad=1)
ax3.set_yticks([0.2, 0.4], labels=['0.2', '0.4'], fontsize=tick_fs)
ax3.set_xticks([0, 5, 10], labels=['0', '5', '10'], fontsize=tick_fs)
ax4.set_yticks([0.05, 0.10, 0.15], labels=['0.05', '0.10', '0.15'], fontsize=tick_fs)
ax3.tick_params(axis='y', pad=1, labelsize=tick_fs)
ax3.tick_params(axis='x', pad=1, labelsize=tick_fs)
ax4.tick_params(axis='y', pad=1, labelsize=tick_fs)
ax3.arrow(x=4, y=0.2, dx=-3, dy=0, width=0.003, head_length=0.2, color=snr_data)
ax3.arrow(x=10.75, y=0.2, dx=3, dy=0, width=0.003, head_length=0.2, color=coherence_data)
a, = ax4.plot(np.array(drive_amps)*10**(3), coherence, linestyle='', marker='.', markersize=6, color=coherence_data)
b, = ax4.plot(np.array(drive_amps)*10**(3), coherence_decay(drive_amps, *popt2), linestyle="-", linewidth=1.5, color=coherence_fit)
c, = ax3.plot(np.array(epsilon)*10**(3), lin_fit(np.array(epsilon), *popt), linestyle='-', linewidth=1.5, color=snr_fit)
d, = ax3.plot(np.array(epsilon)*10**(3), snr, marker='.', linestyle=' ', markersize=6, color=snr_data)
l = ax4.legend([(a, d), (b, c)], [r"Data", r"Fit"],
               handler_map={tuple: HandlerTuple(ndivide=None)}, handletextpad=0.6, frameon=False, handlelength = 1.4, fontsize = tick_fs, labelspacing=0.3, 
               loc='lower left', bbox_to_anchor=(0.28, -0.05))
ax3.annotate("(b)", xy=(-0.15, 0.94), xycoords="axes fraction", fontsize=label_fs)

plt.tight_layout()